In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

In [8]:
# GoogLeNet specific transform
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [10]:
batch_size = 4

train_set = CIFAR10(root='/mnt/e/ml_projects/IN3310/2025/tut_data', train=True, download=True, transform=preprocess)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = CIFAR10(root='/mnt/e/ml_projects/IN3310/2025/tut_data', train=False, download=True, transform=preprocess)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100.0%


Extracting /mnt/e/ml_projects/IN3310/2025/tut_data/cifar-10-python.tar.gz to /mnt/e/ml_projects/IN3310/2025/tut_data
Files already downloaded and verified


In [12]:
import torch.nn as nn
import torch.nn.functional as F

model = torchvision.models.googlenet(num_classes=10)
print(model)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

/mnt/e/ml_projects/in3310/lib/python3.12/site-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [15]:
for epoch in range(2):

    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        # gets inputs: data is a list of [inputs, labels]

        inputs, labels = data[0].to(device), data[1].to(device) # GPU
        
        # zero the parameter grdients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        # loss from three branches
        loss_main = criterion(outputs.logits, labels)
        loss_aux1 = criterion(outputs.aux_logits1, labels)
        loss_aux2 = criterion(outputs.aux_logits2, labels)

        loss = loss_main * 0.4 + loss_aux1 * 0.3 + loss_aux2 * 0.3
        loss.backward()
        optimizer.step()

        # print stats
        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

        


[1,   200] loss: 0.224
[1,   400] loss: 0.213
[1,   600] loss: 0.197
[1,   800] loss: 0.201
[1,  1000] loss: 0.196
[1,  1200] loss: 0.190
[1,  1400] loss: 0.192
[1,  1600] loss: 0.190
[1,  1800] loss: 0.184
[1,  2000] loss: 0.186
[1,  2200] loss: 0.179
[1,  2400] loss: 0.177
[1,  2600] loss: 0.176
[1,  2800] loss: 0.175
[1,  3000] loss: 0.172
[1,  3200] loss: 0.170
[1,  3400] loss: 0.172
[1,  3600] loss: 0.166
[1,  3800] loss: 0.165
[1,  4000] loss: 0.164
[1,  4200] loss: 0.167
[1,  4400] loss: 0.160
[1,  4600] loss: 0.163
[1,  4800] loss: 0.159
[1,  5000] loss: 0.158
[1,  5200] loss: 0.166
[1,  5400] loss: 0.160
[1,  5600] loss: 0.156
[1,  5800] loss: 0.153
[1,  6000] loss: 0.151
[1,  6200] loss: 0.144
[1,  6400] loss: 0.152
[1,  6600] loss: 0.140
[1,  6800] loss: 0.147
[1,  7000] loss: 0.139
[1,  7200] loss: 0.138
[1,  7400] loss: 0.145
[1,  7600] loss: 0.138
[1,  7800] loss: 0.132
[1,  8000] loss: 0.132
[1,  8200] loss: 0.135
[1,  8400] loss: 0.136
[1,  8600] loss: 0.131
[1,  8800] 

In [ ]:
correct = 0
total = 0

# no need to calculate gradients for output here
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)

        outputs = model(images)

        # for testing only using output of main branch
        output_logits = outputs.logits

        _, predicted = torch.max(output_logits.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total}%')


Accuracy of the network on the 10000 test images: 67%


In [18]:
# Check the classes that performed well and thos that didn'test_loader

correct_pred = {class_name: 0 for class_name in classes}
total_pred = {class_name: 0 for class_name in classes}

# no gradient needed here
with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        output_logits = outputs.logits
        _, predictions = torch.max(output_logits, 1)

        # collect right pred for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

# print accuracy for each class
for class_name, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[class_name]
    print(f'Accuracy for class: {class_name:5s} is {accuracy:.1f}%')

Accuracy for class: plane is 79.5%
Accuracy for class: car   is 77.9%
Accuracy for class: bird  is 66.3%
Accuracy for class: cat   is 44.8%
Accuracy for class: deer  is 41.5%
Accuracy for class: dog   is 49.9%
Accuracy for class: frog  is 74.1%
Accuracy for class: horse is 73.8%
Accuracy for class: ship  is 82.1%
Accuracy for class: truck is 83.5%
